<a href="https://colab.research.google.com/github/ZackPashkin/ruGPT3large-simple-web-app-in-colab-/blob/main/ruGPT3large_simple_web_app_in_colab_using_flask_and_ngrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [1]:
!pip3 install transformers==3.5.0
!pip install flask-ngrok
!git clone https://github.com/ZackPashkin/ruGPT3large-simple-web-app-in-colab-.git
!git clone  https://github.com/sberbank-ai/ru-gpts

     |████████████████████████████████| 1.3MB 5.7MB/s 
     |████████████████████████████████| 890kB 34.1MB/s 
     |████████████████████████████████| 2.9MB 38.2MB/s 
     |████████████████████████████████| 1.1MB 39.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=3f41ee5b764cd6bb412ea232ce3c5084e15a6fe04e51ac83393fb80bcc5998b4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Cloning into 'ruGPT3large-simple-web-app-in-colab-'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 30 (delta 6), reused 8 (delta 0), pack-reused 0
Unpacking objects: 100% (30/30), done.
Cloning into 'ru-gpts'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 595 (delta 56),

# Prepare the model

In [2]:
import numpy as np
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [3]:
def load_tokenizer_and_model(model_name_or_path):
  return GPT2Tokenizer.from_pretrained(model_name_or_path), GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()


def generate(
    model, tok, text,
    do_sample=True, max_length=50, repetition_penalty=5.0,
    top_k=5, top_p=0.95, temperature=1,
    num_beams=None,
    no_repeat_ngram_size=3
    ):
  input_ids = tok.encode(text, return_tensors="pt").cuda()
  out = model.generate(
      input_ids.cuda(),
      max_length=max_length,
      repetition_penalty=repetition_penalty,
      do_sample=do_sample,
      top_k=top_k, top_p=top_p, temperature=temperature,
      num_beams=num_beams, no_repeat_ngram_size=no_repeat_ngram_size
      )
  return list(map(tok.decode, out))

In [4]:
# initialize the model
tok, ruGPT3Large = load_tokenizer_and_model("sberbank-ai/rugpt3large_based_on_gpt2")

In [5]:
# check model inference is working
user_input = "рлдрдод"
predictions = generate(ruGPT3Large, tok, user_input, num_beams=5)
predictions[0]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'рлдрдодддддаааа!<s>\nКакие у вас ассоциации с этой картинкой? (см. внутри)\nСоседка пришла в гости, а я не знаю что ей сказать...<s>\nПомогите'

# Run ngrok flask app

In [7]:
# launch app

from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, session, request, redirect 
import os



template_dir = os.path.abspath('/content/ruGPT3large-simple-web-app-in-colab-/templates')
app = Flask(__name__, template_folder=template_dir)
app.secret_key = 'thisIsSecret'
run_with_ngrok(app)


@app.route('/', methods=['POST', 'GET'])
def home():
  return render_template("index.html")

@app.route('/predict',methods = ['POST'])
def predict(): 
  user_input = request.form["guess"] + " "
  print(user_input)
  global tok 
  global ruGPT3Large
  predictions = generate(ruGPT3Large, tok, user_input, num_beams=5)
  assert isinstance(user_input, str), 'Argument of wrong type!'
  if user_input:
    print(user_input)
    text = "ruGPT3 answer: "
    return render_template("predict.html", text=text, pred=predictions[0])

@app.route('/button') 
def get_ses(): 
  return redirect('/')
  	
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://64a12485de12.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [07/Mar/2021 11:15:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Mar/2021 11:15:16] "GET /favicon.ico HTTP/1.1" 404 -
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


j;j;j 


127.0.0.1 - - [07/Mar/2021 11:15:21] "POST /predict HTTP/1.1" 200 -


j;j;j 


127.0.0.1 - - [07/Mar/2021 11:15:25] "GET /button HTTP/1.1" 302 -
127.0.0.1 - - [07/Mar/2021 11:15:25] "GET / HTTP/1.1" 200 -


![alt text](https://drive.google.com/uc?id=1UkilGMiffhC2ElZepgjdrDDBNrsm56UF "demo")